<a href="https://colab.research.google.com/github/Jaeji/AM360Paper/blob/main/Paper_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!Git clone https://github.com/Jaeji/AM360Paper.git

/bin/bash: line 1: Git: command not found
